<a href="https://colab.research.google.com/github/woncoh1/opendata/blob/main/excel2arrow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1. Import libraries

In [ ]:
import ast
import os
import re
import statistics
from toolz import curry
from typing import Dict, Union

import pandas as pd
import ipywidgets as widgets

Colab-specific imports:

In [ ]:
from google.colab import auth, data_table
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive, GoogleDriveFile
from oauth2client.client import GoogleCredentials

# Define constants

In [ ]:
DIRECTORY_GD_ID = '123456789abcdefghijklmn' # Google Drive folder ID
IMPORT_FILENAME = 'data.xlsx'
export_filename = IMPORT_FILENAME.split('.', 1)[0] + '.arrow'

# Define functions

In [ ]:
def list_files(
    dir: str,
) -> Dict[str, GoogleDriveFile]:
    """Get all files from a Google Drive directory."""
    query = {'q': f"parents='{dir}' and trashed=false"}
    files = drive.ListFile(query).GetList()
    return {
        gdf['title']: gdf
        for gdf in files
    }

In [ ]:
def import_excel(
    gdf: GoogleDriveFile,
    sheet_name=0,
    header=0,
    names=None,
    dtype=None,
) -> Union[dict, pd.DataFrame]:
    """Import an Excel file from disk to memory."""
    filename = gdf['title']
    # Google Drive -> Colab disk
    gdf.GetContentFile(filename)
    # Colab disk -> Colab RAM
    dict_or_df = pd.read_excel(
        filename,
        sheet_name=sheet_name,
        header=header,
        names=names,
        dtype=dtype,
    )
    # Clean up Colab disk
    os.system(f'rm {filename}')
    return dict_or_df

In [ ]:
def import_arrow(
    gdf: GoogleDriveFile,
) -> pd.DataFrame:
    """Import an Arrow file from disk to memory."""
    filename = gdf['title']
    # Google Drive -> Colab disk
    gdf.GetContentFile(filename)
    # Colab disk -> Colab RAM
    df = pd.read_feather(filename)
    # Clean up Colab disk
    os.system(f'rm {filename}')
    return df

In [ ]:
def export_data(
    df: pd.DataFrame,
    folder_id: str,
    filename: str,
) -> None:
    """Export an Arrow or Excel file from disk to memory."""
    extension = filename.split('.', 1)[-1]
    # Colab RAM -> Colab disk
    # https://arrow.apache.org/docs/r/reference/write_feather.html
    df.to_feather(filename) if extension == 'arrow' else df.to_csv(filename)
    # Colab disk -> Google Drive
    uploaded = drive.CreateFile({
        'title': filename,
        'parents': [{
            'id': folder_id,
            'kind': 'drive#parentReference',
            'isRoot': 'False',
        }],
    })
    uploaded.SetContentFile(filename)
    uploaded.Upload()

# Authenticate drive

In [ ]:
# Google Colab
auth.authenticate_user()
# Google Drive
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Part 2. Import Excel

In [ ]:
gdf = list_files(DIRECTORY_GD_ID)[IMPORT_FILENAME]
df = import_excel(gdf, header=1, dtype=object)

In [ ]:
df.head()

,PA_DATA_NUM,PA_PAT_ID,PA_AD_NO,PA_PAT_INM,PA_CHART_NO,PA_HOSP_CD,PA_HOSP_NM,PA_DOEN,PA_DOB,PA_AGE,...,F_MUSCULOSKELETAL_DATE_6,F_RRT_6,F_RRT_DATE_6,F_RRT_MODALITY_6,F_RRT_MODALITY_OTH_6,F_DROP_OUT_6,F_DROP_OUT_DATE_6,F_DROP_OUT_CAUSE_6,F_DROP_OUT_CAUSE_OTH_6,F_CMT_6
0,1,01-1000007899,1000013565,YSM,29088772,01,서울성모병원,2015-08-27,1958-12-14,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,01-1000007915,1000013623,BMG,8789979,01,서울성모병원,2015-11-09,1956-05-19,59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,01-1000008460,1000015068,KJT,23494500,01,서울성모병원,2017-06-21,1961-03-01,56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,01-1000008579,1000015693,YHK,20310736,01,서울성모병원,2017-07-26,1962-11-19,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,01-1000008674,1000015977,LIW,8372675,01,서울성모병원,2017-10-02,1947-02-10,70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Export Arrow

In [ ]:
export_data(df, DIRECTORY_GD_ID, export_filename)

# Part 3. Import Arrow

In [ ]:
df = import_arrow(list_files(DIRECTORY_GD_ID)[export_filename])

# Inspect data

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Columns: 2717 entries, PA_DATA_NUM to F_CMT_6
dtypes: object(2717)
memory usage: 9.3+ MB


In [ ]:
df.describe()

,PA_DATA_NUM,PA_PAT_ID,PA_AD_NO,PA_PAT_INM,PA_CHART_NO,PA_HOSP_CD,PA_HOSP_NM,PA_DOEN,PA_DOB,PA_AGE,...,F_MUSCULOSKELETAL_DATE_6,F_RRT_6,F_RRT_DATE_6,F_RRT_MODALITY_6,F_RRT_MODALITY_OTH_6,F_DROP_OUT_6,F_DROP_OUT_DATE_6,F_DROP_OUT_CAUSE_6,F_DROP_OUT_CAUSE_OTH_6,F_CMT_6
count,451,451,451,451,451,451,451,451,451,451,...,0,4,0,0,0,4,0,0,0,0
unique,451,451,451,338,451,8,8,394,447,61,...,0,1,0,0,0,1,0,0,0,0
top,1,01-1000007899,1000013565,KJS,29088772,01,서울성모병원,2021-03-18,1959-01-03,62,...,NaN,N,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN
freq,1,1,1,8,1,130,130,3,2,19,...,NaN,4,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN


In [ ]:
df.dtypes.value_counts()

object    2717
dtype: int64

In [ ]:
df

,PA_DATA_NUM,PA_PAT_ID,PA_AD_NO,PA_PAT_INM,PA_CHART_NO,PA_HOSP_CD,PA_HOSP_NM,PA_DOEN,PA_DOB,PA_AGE,...,F_MUSCULOSKELETAL_DATE_6,F_RRT_6,F_RRT_DATE_6,F_RRT_MODALITY_6,F_RRT_MODALITY_OTH_6,F_DROP_OUT_6,F_DROP_OUT_DATE_6,F_DROP_OUT_CAUSE_6,F_DROP_OUT_CAUSE_OTH_6,F_CMT_6
0,1,01-1000007899,1000013565,YSM,29088772,01,서울성모병원,2015-08-27,1958-12-14,56,...,None,None,None,None,None,None,None,None,None,None
1,2,01-1000007915,1000013623,BMG,8789979,01,서울성모병원,2015-11-09,1956-05-19,59,...,None,None,None,None,None,None,None,None,None,None
2,3,01-1000008460,1000015068,KJT,23494500,01,서울성모병원,2017-06-21,1961-03-01,56,...,None,None,None,None,None,None,None,None,None,None
3,4,01-1000008579,1000015693,YHK,20310736,01,서울성모병원,2017-07-26,1962-11-19,54,...,None,None,None,None,None,None,None,None,None,None
4,5,01-1000008674,1000015977,LIW,8372675,01,서울성모병원,2017-10-02,1947-02-10,70,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,447,09-1000012147,1000029371,KCR,2151758,09,은평성모병원,2021-10-05,1949-06-26,72,...,None,None,None,None,None,None,None,None,None,None
447,448,09-1000012246,1000029749,NSH,2324386,09,은평성모병원,2022-06-05,1940-01-10,82,...,None,None,None,None,None,None,None,None,None,None
448,449,09-1000012319,1000030286,LJB,2297763,09,은평성모병원,2022-02-16,1943-02-22,78,...,None,None,None,None,None,None,None,None,None,None
449,450,09-1000012460,1000031324,JYG,2358422,09,은평성모병원,2022-10-27,1955-09-05,67,...,None,None,None,None,None,None,None,None,None,None
